In [ ]:
import findspark
import plotly.graph_objects as go
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext


findspark.init()
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [ ]:
archivo_entrada = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("entradas/GuatemalaExportsTo.csv")
rddfilter = archivo_entrada.rdd.map(tuple)

In [ ]:
rddExport= rddfilter.map(lambda word: (word[4],word[1]))
rddExport.take(10)

In [ ]:
rddCount=rddExport.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCount.collect())

In [ ]:
rddOrder= sc.parallelize(rddCount.sortBy(lambda a: a[1],False).take(1))
print("Mayor -> %s"%rddOrder.collect())

In [ ]:
#separacion a aeropuertos y funciones
rddNames = rddOrder.map(lambda x: (x[0]))
rddTotals = rddOrder.map(lambda x: (x[1]))
print(rddNames.collect())
print(rddTotals.collect())

In [ ]:
#Creando graficas de barras
fig = go.Figure(data=go.Bar(x=rddNames.collect(),y=rddTotals.collect()))
fig.update_layout(title_text='Pais con mayor exportaciones',title_font_size=30,
                  yaxis=dict(title='Total de exportaciones',title_font_size=25),
                  xaxis=dict(title='Nombre Pais',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('R1_A1.html', auto_open=True)